In [1]:
import pandas as pd
import os
import datetime

In [2]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                         sep="|",dtype=str)
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['sales_square_feet']=store_list['sales_square_feet'].astype(float)
store_list['furniture_sales_square_feet']=store_list['furniture_sales_square_feet'].astype(float)

store_list.head(2)

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,1,BERWICK SHOPPING CENTER,1983-09-15,NaN,2837 WINCHESTER PIKE,NaN,COLUMBUS,OH,43232-9301,-82.914789,39.913636,29217.0,6692.0
1,29,ATHENS SHOPPING CENTER,1984-03-26,NaN,918 E STATE ST,NaN,ATHENS,OH,45701-2188,-82.069765,39.337172,26545.0,5694.0


In [3]:
df_store_type=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/other_input/BL Store List Report from Dom 12.20.19.xlsx")
df_store_type=df_store_type.parse("Store List",dtype=str)
df_store_type=df_store_type[["Store",'SOTF']]
df_store_type=df_store_type[df_store_type['SOTF']=="Yes"]
df_store_type=df_store_type[df_store_type['Store']!="TBD"]
print(len(df_store_type),df_store_type['Store'].nunique())
df_store_type=df_store_type.rename(columns={"Store":"location_id","SOTF":"Store_Type"})
df_store_type['Store_Type']=df_store_type['Store_Type'].replace({"Yes":"SOTF"})
print(df_store_type['Store_Type'].unique())
df_store_type.head(2)

479 479
['SOTF']


,location_id,Store_Type
0,1,SOTF
8,49,SOTF


In [4]:
store_list=pd.merge(store_list,df_store_type,on="location_id",how="left")
writer=pd.ExcelWriter("./BL_store_list_with_SOTF_indicator_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
store_list[pd.notnull(store_list['Store_Type'])].to_excel(writer,"SOTF_addresses",index=False)
store_list.to_excel(writer,"all_address",index=False)
writer.save()